In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
breweries= pd.read_csv('data/breweries.csv')

In [4]:
breweries

,id,name,street_address,zipcode,austin_area,untappd_website
0,0,4th Tap Brewing Cooperative,10615 Metric Boulevard,78758,North,https://untappd.com/4thTap/beer
1,1,Adelbert’s Brewery,2314 Rutland Drive,78758,North,https://untappd.com/adelbertsbrewery/beer
2,2,Austin Beerworks,3001 Industrial Terrace,78758,North,https://untappd.com/austinbeerworks/beer
3,3,Black Star Co-Op,7020 Easy Wind Drive,78752,North,https://untappd.com/BlackStarCo-op/beer
4,4,Brewtorium,6015 Dillard Circle,78752,North,https://untappd.com/Brewtorium/beer
5,5,Celis Brewing,10001 Metric Boulevard,78758,North,https://untappd.com/CelisBrewery/beer
6,6,Circle Brewing,2340 W Braker Lane,78758,North,https://untappd.com/circlebrewing/beer
7,7,Draught House,4112 Medical Parkway,78756,North,https://untappd.com/Draught_House/beer
8,8,Hopsquad Brewing,2307 Kramer Lane,78758,North,https://untappd.com/w/hopsquad-brewing-co/4376...
9,9,Flying Man Brewing,2400 Patterson Industrial Drive,78660,North,https://untappd.com/FlyingManBrewing/beer


In [9]:
#creating list with brewery ids
brewery_id = []
x=0
for brewery in breweries['name']:
    brewery_id += 5 * [x]
    x +=1
#brewery_id


In [7]:
#giant loop to scrape each untappd site for each brewery and append the values into the empty lists
beer_name=[]
beer_type=[]
beer_abv=[]
beer_ibu=[]
beer_rating=[]
beer_number_rating=[]

for url in breweries['untappd_website']:

    browser.visit(url)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    beer = soup.find('div', class_='beer-container')
    beer_name_search = beer.find_all('p', class_='name')[:5]
    beer_type_search = beer.find_all('p', class_='style')[:5]

    beer_abv_search = beer.find_all('div', class_='details-item abv')[:5]
    beer_abv_search2 = beer.find_all('p', class_='abv')[:5]

    beer_ibu_search = beer.find_all('div', class_='details-item ibu')[:5]
    beer_ibu_search2 = beer.find_all('p', class_='ibu')[:5]

    beer_rating_search = beer.find_all('span', class_='num')[:5]

    beer_popular = beer.find_all('div', class_='details-item raters')[:5]
    beer_popular2 = beer.find_all('p', class_='raters')[:5]

    for paragraph in beer_name_search:
        if paragraph.a:
            beer_name.append(paragraph.a.text)

    for paragraph in beer_type_search:
        if paragraph.text:
            beer_type.append(paragraph.text)

    for paragraph in beer_abv_search:
        if paragraph.text:
            beer_abv.append(paragraph.text)
    for paragraph in beer_abv_search2:
        if paragraph.text:
            beer_abv.append(paragraph.text)

    for paragraph in beer_ibu_search:
        if paragraph.text:
            beer_ibu.append(paragraph.text)
    for paragraph in beer_ibu_search2:
        if paragraph.text:
            beer_ibu.append(paragraph.text)

    for paragraph in beer_rating_search:
        if paragraph.text:
            beer_rating.append(paragraph.text)

    for paragraph in beer_popular:
        if paragraph.text:
            beer_number_rating.append(paragraph.text)
    for paragraph in beer_popular2:
        if paragraph.text:
            beer_number_rating.append(paragraph.text)

In [10]:
#check to make sure we get number of values
print(len(brewery_id))
print(len(beer_name))
print(len(beer_type))
print(len(beer_abv))
print(len(beer_ibu))
print(len(beer_rating))
print(len(beer_number_rating))

275
275
275
275
275
275
275


In [11]:
#columns for schema

#brewery_id
#name
#type
#abv
#ibu
#rating
#rating_count
beer_df = pd.DataFrame({'brewery_id':brewery_id,
                       'name':beer_name,
                       'type': beer_type,
                       'abv': beer_abv,
                       'ibu':beer_ibu,
                       'rating':beer_rating,
                       'rating_count':beer_number_rating})

In [12]:
#clean ABV
beer_df['abv']=beer_df['abv'].str.replace('% ABV','')
beer_df['abv']=beer_df['abv'].str.replace('\n','')
#pd.to_numeric()

In [13]:
#convert to number, errors = coerce replace other values as NaN
beer_df['abv']=pd.to_numeric(beer_df['abv'],errors='coerce')

In [14]:
#clean IBU column
beer_df['ibu']=beer_df['ibu'].str.replace(' IBU','')
beer_df['ibu']=beer_df['ibu'].str.replace('\n','')
beer_df['ibu']=pd.to_numeric(beer_df['ibu'],errors='coerce')

In [15]:
#clean Rating Column
beer_df['rating']=beer_df['rating'].str.replace('(','')
beer_df['rating']=beer_df['rating'].str.replace(')','')
beer_df['rating']=pd.to_numeric(beer_df['rating'],errors='coerce')

In [16]:
#clean Rating_Count Column
beer_df['rating_count']=beer_df['rating_count'].str.replace('\n','')
beer_df['rating_count']=beer_df['rating_count'].str.replace(',','')
beer_df['rating_count']=beer_df['rating_count'].str.replace(' Ratings','')
beer_df['rating_count']=pd.to_numeric(beer_df['rating_count'],errors='coerce')

In [17]:
beer_df_clean = beer_df

In [18]:
beer_df_clean.to_csv('data/beer_mostpopular_clean.csv', index = False)

In [19]:
beer_df_clean

,brewery_id,name,type,abv,ibu,rating,rating_count
0,0,Long Walk,IPA - American,7.0,57.0,3.61,5722
1,0,Kung Fu Robot,IPA - American,7.0,43.0,3.65,4870
2,0,Vitality Light Ale,Blonde Ale,5.0,17.0,3.46,2725
3,0,Sun Eater,Gruit / Ancient Herbed Ale,4.8,NaN,3.49,2588
4,0,Highland Scoundrel,Scotch Ale / Wee Heavy,10.6,20.0,3.84,2420
...,...,...,...,...,...,...,...
270,54,Dark Skies,Schwarzbier,4.9,NaN,3.81,1111
271,54,Le Saison,Farmhouse Ale - Saison,6.6,NaN,3.69,760
272,54,Adair,Kölsch,4.5,NaN,3.63,527
273,54,Destination,IPA - American,6.5,45.0,3.62,532


In [20]:
#top rated
#https://untappd.com/4thTap/beer?sort=highest_rated
    
beer_name_r=[]
beer_type_r=[]
beer_abv_r=[]
beer_ibu_r=[]
beer_rating_r=[]
beer_number_rating_r=[]

for url in breweries['untappd_website']:
    url_rate = url + '?sort=highest_rated'
    browser.visit(url_rate)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    beer = soup.find('div', class_='beer-container')
    beer_name_search = beer.find_all('p', class_='name')[:5]
    beer_type_search = beer.find_all('p', class_='style')[:5]

    beer_abv_search = beer.find_all('div', class_='details-item abv')[:5]
    beer_abv_search2 = beer.find_all('p', class_='abv')[:5]

    beer_ibu_search = beer.find_all('div', class_='details-item ibu')[:5]
    beer_ibu_search2 = beer.find_all('p', class_='ibu')[:5]

    beer_rating_search = beer.find_all('span', class_='num')[:5]

    beer_popular = beer.find_all('div', class_='details-item raters')[:5]
    beer_popular2 = beer.find_all('p', class_='raters')[:5]

    for paragraph in beer_name_search:
        if paragraph.a:
            beer_name_r.append(paragraph.a.text)

    for paragraph in beer_type_search:
        if paragraph.text:
            beer_type_r.append(paragraph.text)

    for paragraph in beer_abv_search:
        if paragraph.text:
            beer_abv_r.append(paragraph.text)
    for paragraph in beer_abv_search2:
        if paragraph.text:
            beer_abv_r.append(paragraph.text)

    for paragraph in beer_ibu_search:
        if paragraph.text:
            beer_ibu_r.append(paragraph.text)
    for paragraph in beer_ibu_search2:
        if paragraph.text:
            beer_ibu_r.append(paragraph.text)

    for paragraph in beer_rating_search:
        if paragraph.text:
            beer_rating_r.append(paragraph.text)

    for paragraph in beer_popular:
        if paragraph.text:
            beer_number_rating_r.append(paragraph.text)
    for paragraph in beer_popular2:
        if paragraph.text:
            beer_number_rating_r.append(paragraph.text)

In [21]:
#check to make sure we get number of values
print(len(brewery_id))
print(len(beer_name_r))
print(len(beer_type_r))
print(len(beer_abv_r))
print(len(beer_ibu_r))
print(len(beer_rating_r))
print(len(beer_number_rating_r))

275
275
275
275
275
275
275


In [22]:
beer_df_r = pd.DataFrame({'brewery_id':brewery_id,
                       'name':beer_name_r,
                       'type': beer_type_r,
                       'abv': beer_abv_r,
                       'ibu':beer_ibu_r,
                       'rating':beer_rating_r,
                       'rating_count':beer_number_rating_r})

In [23]:
#clean ABV
beer_df_r['abv']=beer_df_r['abv'].str.replace('% ABV','')
beer_df_r['abv']=beer_df_r['abv'].str.replace('\n','')
#pd.to_numeric()

In [24]:
#convert to number, errors = coerce replace other values as NaN
beer_df_r['abv']=pd.to_numeric(beer_df_r['abv'],errors='coerce')

In [25]:
#clean IBU column
beer_df_r['ibu']=beer_df_r['ibu'].str.replace(' IBU','')
beer_df_r['ibu']=beer_df_r['ibu'].str.replace('\n','')
beer_df_r['ibu']=pd.to_numeric(beer_df_r['ibu'],errors='coerce')

In [26]:
#clean Rating Column
beer_df_r['rating']=beer_df_r['rating'].str.replace('(','')
beer_df_r['rating']=beer_df_r['rating'].str.replace(')','')
beer_df_r['rating']=pd.to_numeric(beer_df_r['rating'],errors='coerce')

In [27]:
#clean Rating_Count Column
beer_df_r['rating_count']=beer_df_r['rating_count'].str.replace('\n','')
beer_df_r['rating_count']=beer_df_r['rating_count'].str.replace(',','')
beer_df_r['rating_count']=beer_df_r['rating_count'].str.replace(' Ratings','')
beer_df_r['rating_count']=pd.to_numeric(beer_df_r['rating_count'],errors='coerce')

In [28]:
beer_df_clean_r = beer_df_r

In [29]:
beer_df_clean_r

,brewery_id,name,type,abv,ibu,rating,rating_count
0,0,Barrel Aged Lawgiver,Stout - Russian Imperial,10.0,NaN,4.31,36
1,0,Barrel Aged Supernaut (Garrison Brothers),Stout - Imperial / Double,13.0,NaN,4.29,17
2,0,Fog of War,IPA - New England,6.8,40.0,4.18,11
3,0,Supernaut Cookies & Cream (Barrel Aged Garriso...,Stout - Imperial / Double,13.0,NaN,4.10,60
4,0,Cask Collective Blood Orange,IPA - American,7.0,NaN,4.10,10
...,...,...,...,...,...,...,...
270,54,Desert Skies,Pilsner - Other,7.5,NaN,4.21,21
271,54,Le Saison Barrel Aged With Peaches And Cherries,Farmhouse Ale - Saison,NaN,NaN,4.21,21
272,54,Rosanna (Blend),Brett Beer,6.5,18.0,4.13,17
273,54,Apricot Sakura,Lambic - Fruit,6.3,NaN,4.12,24


In [31]:
beer_df_clean_r.to_csv('data/beer_toprated_clean.csv', index = False)

In [ ]:
#debug 
test=[]
for url in breweries['Untappd']:
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    beer = soup.find('div', class_='beer-container')
    if beer is None:
        test.append('error')
    else:
        test.append('success')

In [ ]:
#test

In [ ]:
beer_len=[]

for url in breweries['Untappd']:
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    beer = soup.find('div', class_='beer-container')

    beer_name_search = beer.find_all('p', class_='name',limit=5)

    beer_len.append(len(beer_name_search))

In [ ]:
#beer_len

In [ ]:
beer_name=[]
beer_type=[]
beer_abv=[]
beer_ibu=[]
beer_rating=[]
beer_number_rating=[]


url =  breweries['Untappd'][33]
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

beer = soup.find('div', class_='beer-container')

beer_name_search = beer.find_all('p', class_='name',limit=5)

eer_type_search = beer.find_all('p', class_='style',limit=5)

beer_abv_search = beer.find_all('div', class_='details-item abv',limit=5)
beer_abv_search2 = beer.find_all('p', class_='abv',limit=5)[:5]

beer_ibu_search = beer.find_all('div', class_='details-item ibu',limit=5)
beer_ibu_search2 = beer.find_all('p', class_='ibu',limit=5)[:5]

beer_rating_search = beer.find_all('span', class_='num',limit=5)[:5]

beer_popular = beer.find_all('div', class_='details-item raters',limit=5)
beer_popular2 = beer.find_all('p', class_='raters',limit=5)

for paragraph in beer_name_search:
    if paragraph.a:
        beer_name.append(paragraph.a.text)

for paragraph in beer_type_search:
    if paragraph.text:
        beer_type.append(paragraph.text)

for paragraph in beer_abv_search:
    if paragraph.text:
        beer_abv.append(paragraph.text)
for paragraph in beer_abv_search2:
    if paragraph.text:
        beer_abv.append(paragraph.text)

for paragraph in beer_ibu_search:
    if paragraph.text:
        beer_ibu.append(paragraph.text)
for paragraph in beer_ibu_search2:
    if paragraph.text:
        beer_ibu.append(paragraph.text)

for paragraph in beer_rating_search:
    if paragraph.text:
        beer_rating.append(paragraph.text)

for paragraph in beer_popular:
    if paragraph.text:
        beer_number_rating.append(paragraph.text)
for paragraph in beer_popular2:
    if paragraph.text:
        beer_number_rating.append(paragraph.text)

In [ ]:
print(len(beer_name))
print(len(beer_type))
print(len(beer_abv))
print(len(beer_ibu))
print(len(beer_rating))
print(len(beer_number_rating))